# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



Note: you may need to restart the kernel to use updated packages.
nohup eva_server > eva.log 2>&1 &


Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc "https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip"

# unzip
!unzip -n bddtest.zip

--2023-05-11 05:10:24--  https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.


HTTP request sent, awaiting response... 

302 Found
Location: /s/raw/kg1q69ijbhjfecw/bddtest.zip [following]
--2023-05-11 05:10:24--  https://www.dropbox.com/s/raw/kg1q69ijbhjfecw/bddtest.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com/cd/0/inline/B72RAWgO4X6zFjDpF3Vj6MRhWpRDcoTuSRVy0Rht_-xk3mwXirFC1sjYjhUbN3e4r3M_aZ3MCTCWXyEeXqr2xTSx-Dgl5t0UGC4v9r9ufhN_Bkm9ygHReTGORO6AJeA20LVnU7ItywEGk56MH64I5nymyEJyHBLKCW8FEkbMjFUTdg/file# [following]
--2023-05-11 05:10:25--  https://uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com/cd/0/inline/B72RAWgO4X6zFjDpF3Vj6MRhWpRDcoTuSRVy0Rht_-xk3mwXirFC1sjYjhUbN3e4r3M_aZ3MCTCWXyEeXqr2xTSx-Dgl5t0UGC4v9r9ufhN_Bkm9ygHReTGORO6AJeA20LVnU7ItywEGk56MH64I5nymyEJyHBLKCW8FEkbMjFUTdg/file
Resolving uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com (uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601f:15::a27d:90f
Connecting to uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com (uc93de2ee0ab9412360da8ac145d.dl.dropboxusercontent.com)|162.125.5.15|:443... connected.


HTTP request sent, awaiting response... 

...


  inflating: bddtest/videos/bddtest_00a04f658c891f94.mp4  


  inflating: bddtest/videos/bddtest_00a820ef2b98dcf5.mp4  


  inflating: bddtest/videos/bddtest_00a395fed60c0b47.mp4  


   creating: bddtest/info/
  inflating: bddtest/info/bddtest_00a04f65af2ab984.csv  
  inflating: bddtest/info/bddtest_00a9cd6bb39be004.csv  
  inflating: bddtest/info/bddtest_00a2e3ca5c856cde.csv  
  inflating: bddtest/info/bddtest_00a04f658c891f94.csv  
  inflating: bddtest/info/bddtest_00a2e3ca62992459.csv  
  inflating: bddtest/info/bddtest_00a0f008a315437f.csv  
  inflating: bddtest/info/bddtest_00a0f0083c67908e.csv  
  inflating: bddtest/info/bddtest_00a820ef2b98dcf5.csv  


  inflating: bddtest/info/bddtest_00a2f5b6d4217a96.csv  
  inflating: bddtest/info/bddtest_00a395fed60c0b47.csv  


## Load 2 BDD videos

In [4]:
cursor.execute('DROP TABLE IF EXISTS bddtest_1;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('DROP TABLE IF EXISTS bddtest_2;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
response.as_df()

,0
0,Number of loaded VIDEO: 1


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
response.as_df()

""


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
response.as_df()

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
response.as_df()

,CSV,Number of loaded frames
0,bddtest/info/bddtest_00a395fed60c0b47.csv,4355


## Run a Query over the Video dataset


In [7]:
cursor.execute("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
      """)
response = cursor.fetch_all()
response.as_df()

cursor.execute("""SELECT id, Yolo(data) 
                  FROM bddtest_1 
                  WHERE id < 3;""")
response = cursor.fetch_all()
response.as_df()

,bddtest_1.id,yolo.labels,yolo.bboxes,yolo.scores
0,0,"[car, stop sign, car, car, car, car, car, car,...","[[490.0, 333.0, 717.0, 513.0], [344.0, 197.0, ...","[0.93, 0.91, 0.87, 0.82, 0.77, 0.73, 0.73, 0.5..."
1,1,"[car, car, stop sign, car, car, car, car, car,...","[[486.0, 332.0, 716.0, 515.0], [819.0, 346.0, ...","[0.92, 0.87, 0.85, 0.76, 0.74, 0.73, 0.62, 0.5..."
2,2,"[car, stop sign, car, car, car, car, car, car,...","[[482.0, 331.0, 716.0, 520.0], [327.0, 189.0, ...","[0.93, 0.9, 0.87, 0.79, 0.77, 0.74, 0.55, 0.54..."


## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
response.as_df()

,bddtestmeta._row_id,bddtestmeta.id,bddtestmeta.frame_id,bddtestmeta.video_id,bddtestmeta.dataset_name,bddtestmeta.label,bddtestmeta.bbox,bddtestmeta.object_id
0,1,8185,0,4,bddtest,car,"[491.7783, 320.9238, 717.0901, 512.1478]",512
1,2,8186,0,4,bddtest,car,"[928.2679, 298.47574, 1279.5381, 571.17786]",513
2,3,8187,0,4,bddtest,car,"[816.0277, 347.52887, 1029.6998, 474.7344]",514
3,4,8188,0,4,bddtest,car,"[811.87067, 325.91223, 948.2217, 408.2217]",515
4,5,8189,0,4,bddtest,car,"[673.0254, 325.08084, 751.17786, 367.48267]",516
...,...,...,...,...,...,...,...,...
3396,3397,11581,1197,4,bddtest,car,"[554.96533, 335.88916, 588.2217, 378.291]",643
3397,3398,11582,1197,4,bddtest,car,"[693.8106, 350.0231, 734.5497, 395.75058]",644
3398,3399,11583,1197,4,bddtest,car,"[683.83374, 345.03464, 733.71826, 389.93073]",642
3399,3400,11584,1197,4,bddtest,car,"[676.351, 346.69745, 699.6305, 384.11084]",641
